# EEGToolkit 
### A small python package to quickly provide first insights into EEG data from reaction-time delay experiments.

Axel Giottonini, Noah Kleinschmidt, Kalvin Dobler


AdvPy Project, FS 2022

---

The package can be installed via `pip` directly via the TestPyPI package index:

```bash

pip install -i https://test.pypi.org/simple/ EEGToolkit

```

## Groundwork

- Inputs are two datafiles specifying EEG recorded signal data, and accompanying time-event metadata.
    - multiple filetypes are supported (`csv, tsv, txt, npy`)
- Timeframe-Windows are extracted around all specified event timepoints (from the metadata)
- Position-wise T-Tests compare signal differences 
    - between different types of events
    - within the same type of event across replicates, against the event's baseline signal
- Output is a summary figure of the conducted T-Tests


![](../test.png)

The figure layout is adjusted by the number of different types of events that were recorded. (Here we replicated the original metadata several times, so some of the signals are identical...)

![](../test_extended.png)

### Usage

The foundation is an `API` centered on the `EEGData` class which handles file reading, and summarising the output, using class methods and additional functions. 

In [ ]:
import EEGToolkit as eeg

# setup the EEGData object
e = eeg.EEGData( 
                    signal_path = "./data/eeg.txt", 
                    event_path = "./data/events.tsv", 
                    sampling_frequency = 500 
                )

# extract event time-windows
e.extract( start_sec = -0.5, stop_sec = 1.2 )

# perform baseline comparison for each signal
e.baseline()

# change the labelling scales of the data
timescale = 1000 # milliseconds
signalscale = 1000 # millivolts

# generate a summary figure
e.summary( x_scale = timescale, y_scale = signalscale )

For quicker processing we added a `CLI` which can directly be accessed from the terminal as 

```bash
EEGToolkit --eeg ./data/eeg.txt --event ./data/events.tsv -f 500 -s -0.5 -e 1.2
```

For people who don't like working with CLIs so much, we also offer a `GUI` which can be activated using the CLI option `-i`

```bash
EEGToolkit -i
```

### Further ideas
The first improvement that should be done, would be to implement a module for signal preprocessing. Such module could be developed partially with the `scipy` library and the various processing methods it offers. 

To go further with our project, we could extend the EEGData class to support multiple EEG signals, thus working on the signals in different position of the scalp.

With such extension we could also develop topological and tomographical tools using libraries dedicated to the subject such as [`visbrain`](http://visbrain.org/)

Finally we could create statistical tools to compare the responses between subjects.

Out of the scope of the base project, the support for other brain imaging technologies such as fNIRS or fMRI could be implemented. Implementing theese methods would require the use of a GLM instead of the averaging done with EEG and a more sophisticated preprocessing should also be implemented.

Great examples of what is now implemented for neuroimaging is presented on [nipy.org](https://nipy.org/) !

In [ ]:
class EEGDataCollection(EEGData):

    def __init__(self,
                 signal_paths: list[str],
                 positions: list[tuple[float, float, float]] | np.ndarray,
                 event_path: str,
                 sampling_frequency: float,
                 **kwargs) -> None:

        # Verification to assert that the input values are valid...

        self.signals = [super().__init__(el, event_path, sampling_frequency, kwargs) for el in signal_paths]
        self.positions = positions

        # Verification to assert that the elements of the collection are obtained from the same experiment...


    def extract(self,
                start_sec:float,
                stop_sec:float,
                event_type : ( int or tuple or list or np.ndarray ) = None, 
                **kwargs) -> np.ndarray:

        self._data = np.stack([signal.extract(start_sec, stop_sec, event_type, kwargs) for signal in self.signals], axis=-1)
        return self._data
    
    # ...